# Fine-Tune a Generative AI Model for Dialogue Summarization

In [ ]:
!pip install datasets torch transformers evaluate rouge-score loralib peft -q


In [ ]:
from datasets import load_dataset
import numpy as np
import torch
import pandas as pd
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import os
import evaluate

from peft import LoraConfig, get_peft_model, PeftModel
import time
from transformers import TrainingArguments, Trainer

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

from datasets import load_dataset
dataset = load_dataset(huggingface_dataset_name)

dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "google/flan-t5-small"

original_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for p in model.parameters():
        num_params = p.numel()
        all_model_params += num_params
        if p.requires_grad:
            trainable_model_params += num_params

    return {
        "trainable_params": trainable_model_params,
        "all_params": all_model_params,
        "trainable_percent": 100 * trainable_model_params / all_model_params
    }

print(print_number_of_trainable_model_parameters(original_model))

{'trainable_params': 76961152, 'all_params': 76961152, 'trainable_percent': 100.0}


In [ ]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"Summarize the following dialogue:\n{dialogue}"

inputs = tokenizer(prompt, return_tensors="pt")
output_ids = original_model.generate(**inputs, max_new_tokens=64)
output = tokenizer.decode(
    output_ids[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Summarize the following dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
------------------------------------------------------------------------------------

In [ ]:
def tokenize_function(example):
    dialogue = example["dialogue"]
    summary = example["summary"]

    # Build the instruction-style prompt
    prompt = (
        "Summarize the following conversation.\n\n"
        + dialogue
        + "\n\nSummary:\n"
    )

    model_inputs = tokenizer(
        prompt,
        max_length=512,
        truncation=True,
    )

    labels = tokenizer(
        summary,
        max_length=128,
        truncation=True,
    )

    # Add labels to the returned dict
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=False,     # process examples one-by-one
    remove_columns=dataset["train"].column_names
)
tokenized_datasets = tokenized_datasets.with_format("torch")

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [2]:
# subsample the dataset
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (125, 3)
Validation: (5, 3)
Test: (15, 3)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15
    })
})


In [ ]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=1e-4,
    num_train_epochs=1,
    logging_steps=10,
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=original_model,
        padding="longest",
    )
)

In [ ]:
import wandb
wandb.login()
num_epochs = 20
for epoch in range(num_epochs):
    print(f"Starting epoch {epoch+1}/{num_epochs}")

    trainer.train()

    eval_results = trainer.evaluate()
    print(f"Validation results after epoch {epoch+1}: {eval_results['eval_loss']}")

trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Starting epoch 1/20


Step,Training Loss
10,2.046100
20,2.093000
30,1.928900


Validation results after epoch 1: 1.609375
Starting epoch 2/20


Step,Training Loss
10,2.057800
20,2.047700
30,1.925000


...

Step,Training Loss
10,1.914800
20,1.969500
30,1.781200


Validation results after epoch 19: 1.482812523841858
Starting epoch 20/20


Step,Training Loss
10,1.865600
20,1.911700
30,1.771900


Validation results after epoch 20: 1.482812523841858


('./dialogue-summary-training-1763267157/tokenizer_config.json',
 './dialogue-summary-training-1763267157/special_tokens_map.json',
 './dialogue-summary-training-1763267157/spiece.model',
 './dialogue-summary-training-1763267157/added_tokens.json',
 './dialogue-summary-training-1763267157/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load tokenizer and models
# Import T5Tokenizer from transformers
from transformers import T5Tokenizer, AutoModelForSeq2SeqLM

# Define the model path using the config.json path
model_path = output_dir

# Load tokenizer and models
# Use the default T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_path)   # or "t5-base", "t5-large", etc.

# Load the model in a way that is compatible with single-GPU environments
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
instruct_model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']
original_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
)
original_model.to(device)

prompt = f"Summarize the following dialogue:\n{dialogue}"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Move input_ids to the same device as the model
input_ids = input_ids.to(device)

original_model_outputs = original_model.generate(
    input_ids,
    max_new_tokens=64,
)
original_model_text_output = tokenizer.decode(
    original_model_outputs[0],
    skip_special_tokens=True
)

instruct_model_outputs = instruct_model.generate(
    input_ids,
    max_new_tokens=64,
)
instruct_model_text_output = tokenizer.decode(
    instruct_model_outputs[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
You'd like to add a CD-ROM drive to your software.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
#Person1# would like to upgrade the software. #Person2# would like to add a CD-ROM drive.


The instruct model is better, it produces outputs that are more relevant and summary-like than the original model. However, it still misses the teaching aspect (#Person1# teaches #Person2#) and a bit too detailed by mentioning  CD-ROM drive rather than hardware in general, so the improvement is limited. This is expected given the very small fine-tuning dataset.

In [ ]:
#ROUGE metric helps quantify the validity of summarizations produced by models
rouge = evaluate.load("rouge")

In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).input_ids.to(device)

    # Move input_ids to the same device as the model
    input_ids = input_ids.to(device)

    original_model_outputs = original_model.generate(
        input_ids,
        max_length=128
    )
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(
        input_ids,
        max_length=128
    )
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,Is this all correct?,#Person1# is going to take a dictation for the...
1,In order to prevent employees from wasting tim...,Is this all correct?,#Person1# is going to take a dictation for the...
2,Ms. Dawson takes a dictation for #Person1# abo...,Is this all correct?,#Person1# is going to take a dictation for the...
3,#Person2# arrives late because of traffic jam....,Talk to the driver.,#Person1# got stuck in traffic again. #Person2...
4,#Person2# decides to follow #Person1#'s sugges...,Talk to the driver.,#Person1# got stuck in traffic again. #Person2...
5,#Person2# complains to #Person1# about the tra...,Talk to the driver.,#Person1# got stuck in traffic again. #Person2...
6,#Person1# tells Kate that Masha and Hero get d...,M: I think it's a good idea to have a couple o...,#Person1# is not sure what happened to Masha a...
7,#Person1# tells Kate that Masha and Hero are g...,M: I think it's a good idea to have a couple o...,#Person1# is not sure what happened to Masha a...
8,#Person1# and Kate talk about the divorce betw...,M: I think it's a good idea to have a couple o...,#Person1# is not sure what happened to Masha a...
9,#Person1# and Brian are at the birthday party ...,"Brian, how are you?",#Person1# is happy to have a dance with Brian....


In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.08509376386614238), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.08418921232220464), 'rougeLsum': np.float64(0.0865154155307608)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.2288755636461806), 'rouge2': np.float64(0.06099569182733483), 'rougeL': np.float64(0.19244987236205174), 'rougeLsum': np.float64(0.19415436009213588)}


In [ ]:
# Update the path to the CSV file
results_path = "data/dialogue-summary-training-results.csv"
results = pd.read_csv(results_path)

human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.2216686882994889), 'rouge2': np.float64(0.0707492488737373), 'rougeL': np.float64(0.19245630286595683), 'rougeLsum': np.float64(0.192409231638204)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.4041959932817219), 'rouge2': np.float64(0.17064828985299663), 'rougeL': np.float64(0.3267557101191949), 'rougeLsum': np.float64(0.3266766725171105)}


The results show substantial improvement in all ROUGE metrics:

In [ ]:
print("Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL
rouge1: 18.25%
rouge2: 9.99%
rougeL: 13.43%
rougeLsum: 13.43%


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "k", "v", "o"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [ ]:
peft_model = get_peft_model(original_model, lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

{'trainable_params': 688128, 'all_params': 77649280, 'trainable_percent': 0.8862001038515747}


In [ ]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=peft_model,
        padding="longest",
    ),
)

In [ ]:
for epoch in range(num_epochs):
    print(f"Starting epoch {epoch+1}/{num_epochs}")
    peft_trainer.train()
    eval_results = peft_trainer.evaluate()
    print(f"Validation results after epoch {epoch+1}: {eval_results['eval_loss']}")

peft_model_path="./peft-dialogue-summary-checkpoint-local"

peft_trainer.save_model(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Starting epoch 1/20


Step,Training Loss
1,3.015600


Validation results after epoch 1: 2.234375
Starting epoch 2/20


Step,Training Loss
1,2.843800


Validation results after epoch 2: 2.046875
Starting epoch 3/20


...

Step,Training Loss
1,1.242200


Validation results after epoch 19: 1.453125
Starting epoch 20/20


Step,Training Loss
1,1.203100


Validation results after epoch 20: 1.453125


('./peft-dialogue-summary-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-summary-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-summary-checkpoint-local/spiece.model',
 './peft-dialogue-summary-checkpoint-local/added_tokens.json',
 './peft-dialogue-summary-checkpoint-local/tokenizer.json')

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_base = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(peft_model_base)

# Update the PEFT model path
peft_model_path = peft_model_path

peft_model = PeftModel.from_pretrained(
    AutoModelForSeq2SeqLM.from_pretrained(peft_model_base),
    peft_model_path,
    is_trainable=False
)

# Move the entire peft_model to the device
peft_model = peft_model.to(device)


In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

{'trainable_params': 0, 'all_params': 77649280, 'trainable_percent': 0.0}


In [ ]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"Summarize the following dialogue:\n{dialogue}"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Move input_ids to the same device as the model
input_ids = input_ids.to(device)
original_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
).to(device)
original_model_outputs = original_model.generate(
    input_ids,
    max_new_tokens=64
)
original_model_text_output = tokenizer.decode(
    original_model_outputs[0],
    skip_special_tokens=True
)

instruct_model_outputs = instruct_model.generate(
    input_ids,
    max_new_tokens=64
)
instruct_model_text_output = tokenizer.decode(
    instruct_model_outputs[0],
    skip_special_tokens=True
)
peft_model_outputs = peft_model.generate(
    input_ids=input_ids,
    max_new_tokens=64
)
peft_model_text_output = tokenizer.decode(
    peft_model_outputs[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
You'd like to add a CD-ROM drive to your software.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
#Person1# would like to upgrade the software. #Person2# would like to add a CD-ROM drive.
---------------------------------------------------------------------------------------------------
PEFT MODEL: #Person2# considers adding a painting program to his software. #Person1# suggests adding a CD-ROM drive to the software.


In [ ]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    human_baseline_text_output = human_baseline_summaries[idx]

    # Move input_ids to the same device as the model
    input_ids = input_ids.to(device)

    original_model_outputs = original_model.generate(
        input_ids,
        max_length=128
    )
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(
        input_ids,
        max_length=128
    )
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(
        input_ids=input_ids,
        max_length=128
    )
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,Is this the first time you have a dictation fo...,#Person1# is going to take a dictation for the...,#Person2# asks #Person1# to take a dictation f...
1,In order to prevent employees from wasting tim...,Is this the first time you have a dictation fo...,#Person1# is going to take a dictation for the...,#Person2# asks #Person1# to take a dictation f...
2,Ms. Dawson takes a dictation for #Person1# abo...,Is this the first time you have a dictation fo...,#Person1# is going to take a dictation for the...,#Person2# asks #Person1# to take a dictation f...
3,#Person2# arrives late because of traffic jam....,Talk to your boss.,#Person1# got stuck in traffic again. #Person2...,#Person2# suggests starting taking public tran...
4,#Person2# decides to follow #Person1#'s sugges...,Talk to your boss.,#Person1# got stuck in traffic again. #Person2...,#Person2# suggests starting taking public tran...
5,#Person2# complains to #Person1# about the tra...,Talk to your boss.,#Person1# got stuck in traffic again. #Person2...,#Person2# suggests starting taking public tran...
6,#Person1# tells Kate that Masha and Hero get d...,"Kate, you know, I'm not sure.",#Person1# is not sure what happened to Masha a...,#Person2# says that Masha and Hero are getting...
7,#Person1# tells Kate that Masha and Hero are g...,"Kate, you know, I'm not sure.",#Person1# is not sure what happened to Masha a...,#Person2# says that Masha and Hero are getting...
8,#Person1# and Kate talk about the divorce betw...,"Kate, you know, I'm not sure.",#Person1# is not sure what happened to Masha a...,#Person2# says that Masha and Hero are getting...
9,#Person1# and Brian are at the birthday party ...,"Brian, how are you?",#Person1# is going to celebrate #Person1#'s bi...,#Person2# invites #Person1# to celebrate his b...


In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(

    predictions=original_model_summaries,
    references=human_baseline_summaries
)

instruct_model_results = rouge.compute(

    predictions=instruct_model_summaries,
    references=human_baseline_summaries
)

peft_model_results = rouge.compute(

    predictions=peft_model_summaries,
    references=human_baseline_summaries
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.11575268414695844), 'rouge2': np.float64(0.01111111111111111), 'rougeL': np.float64(0.09514258109004686), 'rougeLsum': np.float64(0.09559844081242395)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.21501182632717672), 'rouge2': np.float64(0.055843344505969575), 'rougeL': np.float64(0.1741624493464542), 'rougeLsum': np.float64(0.17546305678335367)}
PEFT MODEL:
{'rouge1': np.float64(0.27695420543696403), 'rouge2': np.float64(0.11300735827590049), 'rougeL': np.float64(0.24506185002736725), 'rougeLsum': np.float64(0.24787301587301586)}


In [ ]:
human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values
peft_model_summaries     = results['peft_model_summaries'].values

original_model_results = rouge.compute(

    predictions=original_model_summaries,
    references=human_baseline_summaries
)

instruct_model_results = rouge.compute(

    predictions=instruct_model_summaries,
    references=human_baseline_summaries
)

peft_model_results = rouge.compute(

    predictions=peft_model_summaries,
    references=human_baseline_summaries
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.2216686882994889), 'rouge2': np.float64(0.0707492488737373), 'rougeL': np.float64(0.19245630286595683), 'rougeLsum': np.float64(0.192409231638204)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.4041959932817219), 'rouge2': np.float64(0.17064828985299663), 'rougeL': np.float64(0.3267557101191949), 'rougeLsum': np.float64(0.3266766725171105)}
PEFT MODEL:
{'rouge1': np.float64(0.39119098357131776), 'rouge2': np.float64(0.15459808342905274), 'rougeL': np.float64(0.31367299251500014), 'rougeLsum': np.float64(0.31360615168633016)}


In [ ]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 16.95%
rouge2: 8.38%
rougeL: 12.12%
rougeLsum: 12.12%


In [ ]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL
rouge1: -1.30%
rouge2: -1.61%
rougeL: -1.31%
rougeLsum: -1.31%


Here you only see a small percentage decrease in the ROUGE metrics vs. full fine-tuned. This implies that the PEFT model's training requires much less computing and memory resources (often just a single GPU) while still able to produce a much better summarization than the baseline model.